# 滑滑梯加密

### Flag 1

Block size = 4 换神仙来都没用，本质上状态是可遍历的。GPT指导能直接出一个PoC，然后面向pwntools改一下就行。

**输入：** 请你阅读algo-slide/algo-slide.py。这里实现了一个DES的变种，把 DES 的 F 函数替换成坚不可摧的 SHA1 散列算法。但是DES本身还是不安全的。请你生成algo-slide/pwndes.py，分析现有算法实现的问题，先看看能不能求出mode=easy的flag。

In [ ]:
import itertools
from typing import Iterable
from pwn import context, process, remote, tube

BLOCK_SIZE = 4
HEX_ALPHABET = b"0123456789ABCDEF"
TOKEN = "PUT_YOUR_TOKEN_HERE"

def iter_candidate_blocks() -> Iterable[bytes]:
    for tpl in itertools.product(HEX_ALPHABET, repeat=4):
        yield bytes(tpl)
    for tpl in itertools.product(HEX_ALPHABET, repeat=3):
        yield bytes(tpl) + bytes([1])
    for tpl in itertools.product(HEX_ALPHABET, repeat=2):
        yield bytes(tpl) + bytes([2, 2])
    for tpl in itertools.product(HEX_ALPHABET, repeat=1):
        yield bytes(tpl) + bytes([3, 3, 3])
    yield bytes([4, 4, 4, 4])


def query_block(io: tube, block: bytes) -> bytes:
    io.sendline(block.hex().encode())
    line = io.recvline()
    if not line:
        raise RuntimeError("Challenge stopped responding")
    return bytes.fromhex(line.strip().decode())


def unpad_pkcs7(data: bytes, block_size: int) -> bytes:
    if not data:
        raise ValueError("Empty data")
    pad_len = data[-1]
    if pad_len == 0 or pad_len > block_size:
        raise ValueError("Invalid padding length")
    if data[-pad_len:] != bytes([pad_len]) * pad_len:
        raise ValueError("Invalid padding")
    return data[:-pad_len]


conn = remote("prob12.geekgame.pku.edu.cn", 10012)
context.log_level = "debug"
conn.recvuntil(b"token:")
conn.sendline(TOKEN.encode())
conn.recvuntil(b"?")
conn.sendline(b"easy")

enc_flag_line = conn.recvline().strip()
enc_flag = bytes.fromhex(enc_flag_line.decode())

block_map = {}
for block in iter_candidate_blocks():
    cipher_block = query_block(conn, block)
    block_map[cipher_block] = block

decoded_blocks = []
for offset in range(0, len(enc_flag), BLOCK_SIZE):
    cipher_block = enc_flag[offset : offset + BLOCK_SIZE]
    try:
        decoded_blocks.append(block_map[cipher_block])
    except KeyError as exc:
        raise RuntimeError(
            f"Missing mapping for block {cipher_block.hex()}"
        ) from exc

padded_plain = b"".join(decoded_blocks)
plain = unpad_pkcs7(padded_plain, BLOCK_SIZE)
flag = bytes.fromhex(plain.decode()).decode()
print(flag)